## imports

In [1]:
# from NuRadioReco.modules import impulsiveSignalReconstructor
from NuRadioReco.modules.RNO_G.dataProviderRNOG import dataProviderRNOG
from NuRadioReco.detector.detector import Detector
from NuRadioReco.framework.parameters import channelParameters as chp
from NuRadioReco.framework.parameters import stationParameters
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.lines import Line2D
import sys, os, argparse
import numpy as np
from datetime import datetime, timezone
from astropy.time import Time
from get_summit_weather_data.weather_reader import WeatherData
sys.path.append(os.path.abspath('/data/user/sanyukta/rno_code'))
from reading.data_reading import *
from functions.functions import *
import warnings
warnings.filterwarnings("ignore")


## root reader

In [2]:

det = Detector('/home/sanyukta/software/source/NuRadioMC/NuRadioReco/detector/RNO_G/RNO_season_2024.json')
# det = Detector(source="rnog_mongo")
det.update(datetime.now())
# det.update(Time("2023-08-01"))

selectors = [lambda event_info : event_info.triggerType != "FORCE"] #remove forced trigger events
reader_kwargs = {
    "mattak_kwargs": {"backend": "uproot"},
    "selectors": selectors  # Pass the list of selectors
}

plt_chs = [13, 16, 19]
root_filepath = '/data/user/sanyukta/rno_data/highwind/0602/station13_run250027_combined.root'
reader = dataProviderRNOG()
reader.begin(root_filepath, det, reader_kwargs=reader_kwargs)
for event in reader.run():
    station = event.get_station()
    # reconstructor.run(event, station, det, use_channels=[13,16,19])

WARNING - 2026-02-13 02:33:56,312 - NuRadioReco.RNO_G.channelBlockOffsetFitter - Trace has large block offsets (>102% of Vrms), removing by fitting.
WARNING - 2026-02-13 02:33:56,369 - NuRadioReco.RNO_G.channelBlockOffsetFitter - Trace has large block offsets (>74% of Vrms), removing by fitting.
WARNING - 2026-02-13 02:33:56,387 - NuRadioReco.RNO_G.channelBlockOffsetFitter - Trace has large block offsets (>97% of Vrms), removing by fitting.
WARNING - 2026-02-13 02:33:56,404 - NuRadioReco.RNO_G.channelBlockOffsetFitter - Trace has large block offsets (>107% of Vrms), removing by fitting.
WARNING - 2026-02-13 02:33:56,422 - NuRadioReco.RNO_G.channelBlockOffsetFitter - Trace has large block offsets (>94% of Vrms), removing by fitting.
WARNING - 2026-02-13 02:33:56,438 - NuRadioReco.RNO_G.channelBlockOffsetFitter - Trace has large block offsets (>72% of Vrms), removing by fitting.
WARNING - 2026-02-13 02:33:56,445 - NuRadioReco.RNO_G.channelBlockOffsetFitter - Trace has large block offsets

## nur reader

In [67]:
det = Detector('/home/sanyukta/software/source/NuRadioMC/NuRadioReco/detector/RNO_G/RNO_season_2024.json')
# det = Detector(source="rnog_mongo")
det.update(datetime.now())
# det.update(Time("2023-08-01"))

nur_filepath = 'nur_files/station12_run250522_combined.nur'
from NuRadioReco.modules.io import eventReader
reader = eventReader.eventReader()
reader.begin(nur_filepath)
tot_evs = 0; zeniths = {}; azimuths = {}; times = {}; 
for event in reader.run():
    tot_evs += 1
    station = event.get_station()
    if station.has_trigger('RADIANT0'):
        color = 'blue'
        marker = '^'
        trig_name = 'RADIANT0'
    elif station.has_trigger('RADIANT1'):
        color = 'darkviolet'
        marker = 'v'
        trig_name = 'RADIANT1'
    elif station.has_trigger('LT'):
        color = 'red'
        marker = 'o'
        trig_name = 'LT'
    trigger = station.get_triggers()[trig_name]
    station_time = station.get_station_time().unix

    try:
        zenith = station[stationParameters.zenith]
        azimuth = station[stationParameters.azimuth]
    except: 
        continue
    if zenith:
        zeniths[event.get_id()] = np.degrees(zenith)
        azimuths[event.get_id()] = np.degrees(azimuth)
        times[event.get_id()] = station_time
        print(f"Event {event.get_id()}: Zenith = {np.degrees(zenith):.2f} deg, Azimuth = {np.degrees(azimuth):.2f} deg, trigger = {trig_name}, time = {station_time}")    

Event 7156: Zenith = 56.82 deg, Azimuth = 72.74 deg, trigger = RADIANT0, time = 1757350356.2583344


In [ ]:
chs = [13]
fig, ax = plt.subplots(3,1)
print(f'Total reconstructed events: {len(zeniths)} out of {tot_evs}')
for event in reader.run():
    if event.get_id() not in zeniths.keys():
        continue
    if event.get_id()==6906:
        import NuRadioReco.modules.channelBandPassFilter
        channelBandPassFilter = NuRadioReco.modules.channelBandPassFilter.channelBandPassFilter()
        station = event.get_station()
        print(station.get_triggers())
        # channelBandPassFilter.run(event, station, det, passband=[200 * units.MHz, 700 * units.GHz], filter_type='butter', order=2)
        for ch in chs:
            channel = station.get_channel(ch)
            ax[0].plot(channel.get_times(), channel.get_trace(), label=f'Ch{ch}')
            ax[1].plot(channel.get_frequencies(), np.abs(channel.get_frequency_spectrum()), label=f'Ch{ch}')
            ax[2].plot(channel.get_times(), channel.get_hilbert_envelope())
            corr = calc_dispersivity(channel.get_hilbert_envelope())
            scale, fit = fit_landau(channel.get_hilbert_envelope())
            ax[2].plot(channel.get_times(), fit, '--', color='r', label=f'Dispersivity = {np.sum(corr):.2f}\nLandau scale = {scale:.2f}')
            print(f'Event {event.get_id()}: Dispersivity = {np.sum(corr):.2f}, Landau scale = {scale:.2f}, snr = {channel[chp.SNR]['peak_2_peak_amplitude_split_noise_rms']:.2f}')
        ax[1].legend(title=f'event_id:{event.get_id()}\ntime:{datetime.fromtimestamp(times[event.get_id()],tz=timezone.utc).strftime('%H:%M:%S')}\nzenith:{zeniths[event.get_id()]:.2f} azimuth:{azimuths[event.get_id()]:.2f}', loc=1, framealpha=0.5)
        ax[2].legend()
ax[1].set_xlim(0.04,0.7)


In [69]:
rpm = pd.read_csv('RPM-2026-02-13-08_23_04.csv')
rpm['unix_time'] = pd.to_datetime(rpm['Time']).astype('int64') // 10**9

In [ ]:
wd = WeatherData(years=[2025], use_official=True, use_unofficial=True)
chs = [13,16,19]
fig, ax = plt.subplots(3,1, sharex=True, figsize=(12, 12), gridspec_kw={'hspace': 0.05})
for index, ch in enumerate(chs):
    scales = {}; ws = {}; snr = {}; turbine={}
    for event in reader.run():
        if event.get_id() not in zeniths.keys():
            continue
        station = event.get_station()
        channel = station.get_channel(ch)
        corr = calc_dispersivity(channel.get_hilbert_envelope())
        scale, fit = fit_landau(channel.get_hilbert_envelope())
        if scale is np.nan:
            continue
        
        rpm_value = rpm.loc[(rpm['unix_time'] - station.get_station_time().unix).abs().idxmin()]['Station 12']
        turbine[station.get_station_time().unix] = float(rpm_value.replace(' rpm', ''))
        ws[station.get_station_time().unix] = wd.getWindSpeed(station.get_station_time())
        scales[station.get_station_time().unix] = scale
        snr[station.get_station_time().unix] = channel[chp.SNR]['peak_2_peak_amplitude_split_noise_rms']

    scatter = ax[index].scatter(scales.keys(), scales.values(), c=turbine.values(), cmap='viridis', s=[5*v for v in snr.values()], edgecolors='midnightblue')
    fig.colorbar(scatter, label='RPM', ax=ax[index])
    ax[index].grid()
    ax[index].hlines(y=30, xmin=list(scales.keys())[0], xmax=list(scales.keys())[-1], colors='orange', linestyles='dashed')
    count_greater_30 = sum(1 for v in scales.values() if v > 30)
    ax[index].legend(title=f"{nur_filepath.split('_')[2]} ch:{ch}\nDispersed events: {count_greater_30} / {len(scales)}", framealpha=0.5)

ax[2].set_ylim(0, 200)
ax[1].set_ylim(0, 200)
ax[2].set_xlabel('Time (unix)')
ax[1].set_ylabel('Landau Scale Parameter')
plt.tight_layout()

In [16]:
# convert from unix time to datetime
datetime.fromtimestamp(list(scales.keys())[0], tz=timezone.utc).strftime('%Y-%m-%d %H:%M')
# datetime.fromtimestamp(list(scales.keys())[-1], tz=timezone.utc).strftime('%H:%M')

'2025-09-08 19:03'

## skymap

In [ ]:
tot_evs = 0
zeniths = {}
azimuths = {}
times = {}
colors_dict = {}
markers_dict = {}

det = Detector('RNO_season_2023_rot5.json')
# det = Detector(source="rnog_mongo")
det.update(datetime.now())
solar1 = det.get_relative_position(12, 51, mode='device')
solar2 = det.get_relative_position(12, 52, mode='device')

# Convert solar panel positions to zenith and azimuth
# Assuming solar positions are [x, y, z] relative to station center
solar1_zenith = np.arctan2(np.sqrt(solar1[0]**2 + solar1[1]**2), solar1[2])
solar1_azimuth = np.arctan2(solar1[1], solar1[0])

solar2_zenith = np.arctan2(np.sqrt(solar2[0]**2 + solar2[1]**2), solar2[2])
solar2_azimuth = np.arctan2(solar2[1], solar2[0])

# Create polar plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

# Plot solar panel positions
ax.scatter(solar1_azimuth, np.degrees(solar1_zenith), 
          c='gold', marker='*', s=500, 
          alpha=1.0, edgecolors='black', linewidths=2,
          label='Solar Panel 1', zorder=10)
ax.scatter(solar2_azimuth, np.degrees(solar2_zenith), 
          c='gold', marker='*', s=500, 
          alpha=1.0, edgecolors='black', linewidths=2,
          label='Solar Panel 2', zorder=10)

for event in reader.run():
    tot_evs += 1
    station = event.get_station()
    
    # Determine trigger type and assign color/marker
    if station.has_trigger('RADIANT0'):
        color = 'blue'
        marker = '^'
        trig_name = 'RADIANT0'
    elif station.has_trigger('RADIANT1'):
        color = 'darkviolet'
        marker = 'v'
        trig_name = 'RADIANT1'
    elif station.has_trigger('LT'):
        color = 'red'
        marker = 'o'
        trig_name = 'LT'
    else:
        continue
    
    trigger = station.get_triggers()[trig_name]
    station_time = station.get_station_time().unix

    try:
        zenith = station[stationParameters.zenith]
        azimuth = station[stationParameters.azimuth]
    except: 
        continue
    
    if zenith:
        event_id = event.get_id()
        zeniths[event_id] = np.degrees(zenith)
        azimuths[event_id] = np.degrees(azimuth)
        times[event_id] = station_time
        colors_dict[event_id] = color
        markers_dict[event_id] = marker
        
        # Plot on skymap
        ax.scatter(azimuth, np.degrees(zenith), 
                  c=color, marker=marker, s=100, 
                  alpha=0.6, edgecolors='black', linewidths=0.5)
        
        print(f"Event {event_id}: Zenith = {np.degrees(zenith):.2f} deg, Azimuth = {np.degrees(azimuth):.2f} deg, trigger = {trig_name}, time = {station_time}")

# Configure polar plot
ax.set_theta_zero_location('N')  # 0° at top (North)
ax.set_theta_direction(-1)  # Clockwise
ax.set_ylim(0, 90)  # Zenith from 0° (vertical) to 90° (horizon)
ax.set_title(f'Station 12 - Reconstructed Sources\n(events: {len(zeniths)} / {tot_evs})', 
             fontsize=14, pad=20)

# Create legend
legend_elements = [
    Line2D([0], [0], marker='^', color='w', markerfacecolor='blue', markersize=10, 
           markeredgecolor='black', label='RADIANT0'),
    Line2D([0], [0], marker='v', color='w', markerfacecolor='darkviolet', markersize=10, 
           markeredgecolor='black', label='RADIANT1'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, 
           markeredgecolor='black', label='LT'),
    Line2D([0], [0], marker='*', color='w', markerfacecolor='gold', markersize=15, 
           markeredgecolor='black', label='Solar Panels')
]
ax.legend(handles=legend_elements, loc='upper right', fontsize=12)

plt.tight_layout()
plt.savefig('skymap_station12.png', dpi=150, bbox_inches='tight')
print(f"\nSaved skymap with {len(zeniths)} reconstructed events out of {tot_evs} total events")
print(f"Solar Panel 1: Zenith = {np.degrees(solar1_zenith):.2f}°, Azimuth = {np.degrees(solar1_azimuth):.2f}°")
print(f"Solar Panel 2: Zenith = {np.degrees(solar2_zenith):.2f}°, Azimuth = {np.degrees(solar2_azimuth):.2f}°")